## Install

In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl
%pip install huggingface-hub
%pip install --upgrade openai
%pip install --upgrade pydantic
%pip install tiktoken
%pip install numpy
%pip install transformers_stream_generator einops

In [2]:
%pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/e9/07/63a6e890c9b998a6318b46c2a34377fd1a3e01a94c427d82bfb2472b7c16/streamlit-1.30.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for validators<1,>=0.2 from https://files.pythonhosted.org/packages/3a/0c/785d317eea99c3739821718f118c70537639aa43f96bfa1d83a71f68eaf6/validators-0.22.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


## Import

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, logging #, BitsAndBytesConfig,HfArgumentParser,TrainingArguments
# from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
# wandb
from datasets import load_dataset
# from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from transformers import pipeline

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_gemini = user_secrets.get_secret("GEMINI_API_KEY")
# secret_wandb = user_secrets.get_secret("wandb")

In [6]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
base_model = "openchat/openchat-3.5-1210"
new_model = "BK-BigAI-Math"
model_hotamath_path = "/kaggle/working/BK-BigAI-Math"

In [8]:
import os
import json
import re
import time
import math
import numpy as np
import numpy
from math import *

## Promt theme

In [9]:
DEFAULT_PAD_TOKEN = "<|pad_0|>" # "[PAD]" "<|pad_0|>"
DEFAULT_EOS_TOKEN = "<|end_of_turn|>" # "<|end_of_turn|>" # "</s>" "<|endoftext|>" "<|end_of_turn|>"
DEFAULT_BOS_TOKEN = "<s>" # "<s>" "<|endoftext|>"
DEFAULT_UNK_TOKEN = "<unk>" # "<unk>" "<|endoftext|>"
DEFAULT_BOI_TOKEN = "[INST]" # "Human:" # "[INST]" "<|human|>" "<|end_of_turn|>"
DEFAULT_EOI_TOKEN = "[/INST]" # "Assistant:" # "[/INST]" "<|human|>" "<|end_of_turn|>"
PROMPT_DICT = {
    ## Implement LLM
    "prompt_formula_i": (
        "Math Correct User: write Explanation and Expression\n"
        "Write a explanation for solution and a python expression that caculate true answer of question. "
        "Expression must be runable python expression, must in one line and only using library math, numpy, datetime, itertools"
        "\n\n"
        "## Question:\n{question}\n\n"
        "" + DEFAULT_EOI_TOKEN + ""
        "Math Correct Assistant:\n"
        "## Explanation:\n{explanation}\n\n"
        "## Expression:\n{formula}"
        "" + DEFAULT_EOS_TOKEN  
    ),
    "prompt_formula_run_i": (
        "Math Correct User: write Explanation and Expression\n"
        "Write a explanation for solution and a python expression that caculate true answer of question. "
        "Expression must be runable python expression, must in one line and only using library math, numpy, datetime, itertools"
        "\n\n"
        "## Question:\n{question}\n\n"
        "" + DEFAULT_EOI_TOKEN + ""
        "Math Correct Assistant:"
    ),
     "prompt_formula_run_previous_i": (
        "Math Correct User: write Explanation and fix Expression\n"
        "Write a explanation for solution and fix python expression that caculate true answer of question. "
        "Expression must be runable python expression, must in one line and only using library math, numpy, datetime, itertools. "
        "If expression result is an error or caculate wrong answer then must fix it."
         "\n\n"
        "## Previous expression:\n{formula} = {outformula}\n"
        "## Question:\n{question}\n\n"
        "" + DEFAULT_EOI_TOKEN + ""
        "Math Correct Assistant:"
    ),
    "prompt_input_i": (
        "Math Correct User: write true Answer and explantion\n"
        "Write a explanation to solve question and write true answer to question.\n\n"
        "## Question:\n{question}\n\n"
        "## Expression caculate answer:\n{formula} = {outformula}"
        "" + DEFAULT_EOI_TOKEN + ""
        "Math Correct Assistant:\n"
        "## Explanation:\n{explanation}\n\n"
        "## Answer:\n{answer}"
        "" + DEFAULT_EOS_TOKEN
    ),
    "prompt_input_run_i": (
        "Math Correct User: write true Answer and explantion\n"
        "Write a explanation to solve question and write true answer to question.\n\n"
        "## Question:\n{question}\n\n"
        "## Expression caculate answer:\n{formula} = {outformula}"
        "" + DEFAULT_EOI_TOKEN + ""
        "Math Correct Assistant:"
    ),
}

In [10]:
timeGlobal = 0
def startTime():
    global timeGlobal
    timeGlobal = time.time()
def getTime():
    return (time.time() - timeGlobal)

In [11]:
def ApplyPromptTemplate(data_arg, typeP = "prompt_formula"):
    return PROMPT_DICT[typeP].format(**data_arg)

## Load modal

In [12]:
model_name_or_path = "hotamago/HotaMath-OpenMath-2023-01-19-ElementaryMathematics"

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    model_max_length=1024,
    padding_side="right",
    trust_remote_code=True
)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/885 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Pipe model

In [14]:
pipeGenEx = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
#     do_sample=False,
    do_sample=True,
    temperature=0.01,
    top_p=0.4,
    top_k=40,
#     repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
pipeGenExStick = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.01,
    top_p=1.0,
#     top_k=50,
    repetition_penalty=1.3,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
pipeGenAns = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
#     do_sample=False,
    do_sample=True,
    temperature=0.01,
    top_p=0.4,
    top_k=40,
#     repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
pipeGenAnsStick = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.01,
    top_p=1.0,
#     top_k=80,
    repetition_penalty=1.3,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

## Auto function run temple

In [15]:
# Implement
import random

def niceValueToCompire(x):
    y = re.findall("^[\n ]*([a-z])\. ?", x.strip().lower())
    if len(y) == 0:
        return x.strip().lower()
    return y[0].strip()

def autoLLMFormatExpVsFormula_i(inp, debug=False):
    prompt_template = ApplyPromptTemplate({
        "question": inp['question'],
    }, "prompt_formula_run_i")
    
    res = pipeGenEx(prompt_template)[0]['generated_text']
    if debug:
        print(res)
        
    x = re.findall("##[ ]*[Ee]xplanation(?:.*):?\n([\s\S]*?)\n\n##[ ]*[Ee]xpression(?:.*):?\n(.*)", res)
    
    if len(x) == 0:
        res = pipeGenExStick(prompt_template)[0]['generated_text']
        x = re.findall("##[ ]*[Ee]xplanation(?:.*):?\n([\s\S]*?)\n\n##[ ]*[Ee]xpression(?:.*):?\n(.*)", res)

    if len(x) == 0:
        print("## Res:\n{0}".format(res))
        return False, None, None
    
    return True, x[-1][0], x[-1][1]

def autoLLMFormatExpVsFormulaPrev_i(inp, prevExplanation, prevFormula, debug=False):
#     print(inp)
    try:
        outformula = eval(prevFormula)
    except Exception as ist:
        outformula = "Invail expression: {0}".format(str(ist))
        
    prompt_template = ApplyPromptTemplate({
        "question": inp['question'],
        "explanation": prevExplanation,
        "formula": prevFormula,
        "outformula": outformula,
    }, "prompt_formula_run_previous_i")
    
    res = pipeGenEx(prompt_template)[0]['generated_text']
    if debug:
        print(res)
        
    x = re.findall("##[ ]*[Ee]xplanation(?:.*):?\n([\s\S]*?)\n\n##[ ]*[Ee]xpression(?:.*):?\n(.*)", res)
    
    if len(x) == 0:
        res = pipeGenExStick(prompt_template)[0]['generated_text']
        x = re.findall("##[ ]*[Ee]xplanation(?:.*):?\n([\s\S]*?)\n\n##[ ]*[Ee]xpression(?:.*):?\n(.*)", res)

    if len(x) == 0:
        print("## Res:\n{0}".format(res))
        return False, None, None
    
    return True, x[-1][0], x[-1][1]

def autoLLMFormatAnswer_i(inp, debug=False):
    # init random choice
    question = inp['question']
    
    # Run get first formula
    ok, explanation, formula = autoLLMFormatExpVsFormula_i(inp, debug=debug)
    
    if ok == False:
        print(f"Error, no 01: {inp}")

    # Run get n prev formula
#     i = 0
#     while i < 2:
#         ok, explanation, formula = autoLLMFormatExpVsFormulaPrev_i(inp, explanation, formula, debug=debug)
#         if ok == False:
#             continue
#         i += 1
#             print(f"Error, no 01: {inp}")
            
    try:
        formula = formula.strip()
    except:
        formula = "No expression"
    
    try:
        outformula = eval(formula)
    except Exception as ist:
        outformula = "Invail expression"
        print(ist)
        
    prompt_template = ApplyPromptTemplate({
        "question": question,
        "explanation": explanation,
        "formula": formula,
        "outformula": outformula,
    }, "prompt_input_run_i")
    
    res = pipeGenAns(prompt_template)[0]['generated_text']
    
    if debug:
        print(res)
        
    x = re.findall("##[ ]*(?:[Aa]nswer|[Đđ]áp số|[Tt]he answer)(?:.*):?[\n ](.+)", res)
    
    if len(x) < 1:
        res = pipeGenAnsStick(prompt_template)[0]['generated_text']
        x = re.findall("##[ ]*(?:[Aa]nswer|[Đđ]áp số|[Tt]he answer)(?:.*):?[\n ](.+)", res)

    if len(x) < 1:
        print("Error, no Answer 1: {0}".format(res))
        raise Exception("Error, no Answer 1")
    
#     x = niceValueToCompire(x[-1])
    x = x[-1]
    
    if x is None:
        print("Error, no Answer 2: {0}".format(res))
        raise Exception("Error, no Answer 2")
    
    return explanation, formula, x
    
    return "wtf"

## Implement

In [16]:
import math
import numpy
import datetime
import itertools

In [17]:
def AutoInp2OutImplement(question):
    explanation, formula, answer = autoLLMFormatAnswer_i({"question": question}, False)
    outformula = ""
    try:
        outformula = " = " + str(eval(formula))
    except:
        pass
    return explanation, formula, outformula, answer

In [18]:
def AutoRawInput2TextOutImplement(question):
    explanation, formula, outformula, answer = AutoInp2OutImplement(question)
    return (
        "## Explanation:\n{0}\n\n"
        "## Formula:\n{1}{2}\n\n"
        "## Answer:\n{3}\n\n"
    ).format(explanation, formula, outformula, answer)

In [22]:
%%writefile main.py

import streamlit as st
import random
import time

def main():
    st.title("HotaMath - Elementary Mathematics")

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat messages from history on app rerun
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Accept user input
    if prompt := st.chat_input("Nhập câu hỏi..."):
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})
        # Display user message in chat message container
        with st.chat_message("user"):
            st.markdown(prompt)

        # Display assistant response in chat message container
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            full_response = ""
            assistant_response = AutoRawInput2TextOutImplement(prompt)
            # Simulate stream of response with milliseconds delay
            for chunk in assistant_response.split():
                full_response += chunk + " "
                time.sleep(0.05)
                # Add a blinking cursor to simulate typing
                message_placeholder.markdown(full_response + "▌")
            message_placeholder.markdown(full_response)
        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": full_response})

if __name__ == '__main__':
    main()

Writing main.py


## Public server to ngrok

In [28]:
!sudo apt install curl -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcurl4 libcurl4-openssl-dev
Suggested packages:
  libcurl4-doc libidn11-dev libkrb5-dev libldap2-dev librtmp-dev libssh2-1-dev
The following packages will be upgraded:
  curl libcurl4 libcurl4-openssl-dev
3 upgraded, 0 newly installed, 0 to remove and 78 not upgraded.
Need to get 869 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libcurl4-openssl-dev amd64 7.81.0-1ubuntu1.15 [386 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 curl amd64 7.81.0-1ubuntu1.15 [194 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libcurl4 amd64 7.81.0-1ubuntu1.15 [289 kB]
Fetched 869 kB in 0s (2117 kB/s) 

(Reading database ... 114840 files and directories currently installed.)
Preparing to unpack .../libcurl4-openssl-de

In [29]:
!curl -L --output cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared.deb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 16.9M  100 16.9M    0     0  31.4M      0 --:--:-- --:--:-- --:--:-- 31.4M
Selecting previously unselected package cloudflared.
(Reading database ... 114840 files and directories currently installed.)
Preparing to unpack cloudflared.deb ...
Unpacking cloudflared (2024.1.3) ...
Setting up cloudflared (2024.1.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [30]:
!sudo cloudflared service install eyJhIjoiNDE2ZDU0MTMwNmViYTJmNWM4NDMzNzIyOGM4NzQ1M2YiLCJ0IjoiOTkzMDYzZTEtOGI4Yi00ZjhkLTgyNDUtZjk5NTQyM2ZkNmM3IiwicyI6Ik0yTXpPRFF3TVdVdFlqUmhaQzAwTnpRM0xXRXlOalV0WTJRM09UZzRNamcwTlRSaiJ9

2024-01-19T14:54:00Z INF Using SysV
2024-01-19T14:54:00Z INF Linux service for cloudflared installed successfully


In [2]:
!streamlit run ./main.py --server.port 8051

/bin/bash: line 1: streamlit: command not found


## Local run

In [ ]:
print(AutoRawInput2TextOutImplement(""))